In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver

# 마우스 자동 스크롤
from selenium.webdriver import ActionChains
# 웹 동작 속도에 따른 수동 딜레이
import time
import pandas as pd

In [2]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
# chrome driver
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.implicitly_wait(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [5]:
name_list = []
eng_name_list = []
price_list = []

In [6]:
# 파싱 함수 정의
def parsing(type_code, num):
    for i in range(1, num+1):
        URL = "https://www.caffe-pascucci.co.kr/product/productList.asp?typeCode="+ type_code + "000" + str(i) + "0"
        driver.get(URL)
        menu = BeautifulSoup(urlopen(URL), "html.parser")
        time.sleep(0.5)

        html = urlopen(URL)
        Menu = BeautifulSoup(html, "html.parser")

        # 한국어 메뉴명
        name = Menu.select('figcaption > h2')
        for n in name:
            name_list.append(n.get_text())

        # 영어 메뉴명
        engname = Menu.select('figcaption > p.titleEng')
        for en in engname:
            eng_name_list.append(en.get_text())

        elements_list = driver.find_elements_by_class_name('product')
        for a in elements_list:
            # 자동 클릭
            a.click()

            # 가격 정보 리스트에 추가
            price = driver.find_element_by_class_name('sizeInfo').text.split(' ')

            # 문자열 슬라이싱
            if len(price) > 1:
                price_list.append(price[1])
            else:
                price_list.append(price[0])

            # 여유시간
            time.sleep(0.5)

            # 닫기
            div_class = driver.find_element_by_class_name('proDtCon')
            close_button = div_class.find_element_by_tag_name('button')
            close_button.click()

            # 스크롤 올리기
            action = ActionChains(driver)
            upper_tag = driver.find_element_by_class_name('location')
            action.move_to_element(upper_tag).perform()

            # 여유시간
            time.sleep(0.5)

In [7]:
parsing("001", 4)
parsing("002", 5)
parsing("003", 5)
driver.quit()

In [8]:
# dataframe 변환
df = pd.DataFrame({'name':name_list, 'eng_name':eng_name_list, 'price':price_list})
print(df)

            name               eng_name   price
0          에스프레소               Espresso  3,600원
1      에스프레소 도피오        Espresso Doppio  4,100원
2     에스프레소 마끼아또     Espresso Macchiato  4,100원
3           아포가또               Affogato  5,500원
4      오리지널 드립커피   Original Drip Coffee  4,200원
..           ...                    ...     ...
173       피넛 젤라또          Peanut Gelato  4,000원
174       라떼 젤라또   Fior Di Latte Gelato  4,000원
175       딸기 젤라또      Strawberry Gelato   4,000
176      요거트 젤라또          Yogurt Gelato  4,000원
177  아마레나 체리 젤라또  Amarena Cherry Gelato  4,000원

[178 rows x 3 columns]
